First the detector, laser and motor are setup along with the bluesky environment from the ophyd_inter_setup file.
Due to the different file structure of the Jupyter notebook the read path must be updated.
An error related to caRepeater will likely appear, this is not a concern.

In [ ]:
from ophyd_inter_setup import *
det.hdf1.read_path_template =  "/home/jovyan/data/%Y/%m/%d/"  

The synced scan can then be run. This example scans in 11 steps between -10 and +10.

In [ ]:
uids = RE(pulse_sync([det], motor1, laser1, -10, 10, 11))

To access the data stored in the databroker catalog run the following:

In [ ]:
run = catalog[uids[0]] #Accesses the run based on its uid, the most recent run can also be accessed as catalog[-1]
data = run.primary.read()
data

The image data is accessed as shown below.

In [ ]:
image = data["det_image"]
frame = image[0][0]  # Index 1 refers to the time of the image and the second index refers to the frame number
frame.plot.pcolormesh()